# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

In [137]:
import numpy as np
import pandas as pd

In [138]:
data = pd.read_csv('final project 2024.csv')

In [139]:
data.head(10)

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
5,6,49
6,7,66
7,8,29
8,9,55
9,10,39


In [140]:
s = 10 # sales price
c_m = 8 # cost mexico
c_c = 7.25 # cost china
i = 0.01 # interest rate
ini_bal = 0 # initial balance

## Single Sourcing: China  

In [141]:
Days_loop = np.arange(2, 10)

### Cumulative Average Forecast

In [142]:
data

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
...,...,...
9995,9996,68
9996,9997,70
9997,9998,74
9998,9999,52


In [143]:
def China_cul_avg_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, days):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []
    # forecast_lst = [0]
    # demand_lst = []
    # error_lst =[]
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        # demand_lst.append(current_demand)
        
        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
            # forecast_lst.append(cum_avg_demand)
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand
            # forecast_lst.append(cum_avg_demand)

        # if len(forecast_lst) > days:
        #     avg_lead_time_demand = np.sum(forecast_lst[-days:])
        #     avg_demand = np.sum(demand_lst[-days:])
        #     error_lst.append(avg_demand - avg_lead_time_demand)
        # else:
        #     avg_lead_time_demand = np.sum(forecast_lst[-len(forecast_lst):])
        #     avg_demand = np.sum(demand_lst[-len(demand_lst):])
        #     error_lst.append(avg_demand - avg_lead_time_demand)
        
        # Check if any orders are arriving in this period
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]
                    
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if cum_avg_demand*days >= inventory:
            order_quantity = int(cum_avg_demand*days - inventory)
        else:
            order_quantity = 0
        
        if order_quantity > 0:
            orders.append((period + 4, order_quantity)) 

        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [144]:
max_balance = 0
max_day = 0

for day in Days_loop:
    balance = China_cul_avg_n_days(data, s, c_c, i, ini_bal, day)
    if balance > max_balance:
        max_balance = balance
        max_day = day
print(f'Current {max_day}, Total Balanced: {max_balance}')

Current 2, Total Balanced: 4.80241051738797e+47


In [145]:
max_balance_model_1 = max_balance
max_day_model_1 = max_day

max_balance_model_1, max_day_model_1

(4.80241051738797e+47, 2)

In [146]:
def China_moving_avg_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, day):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []
    # forecast_lst = [0]
    # demand_lst = []
    # error_lst =[]
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        # demand_lst.append(current_demand)
        
        # Calculate cumulative average demand up to the current period
        if period >= moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] > period - moving_avg_window) & (data['period'] <= period), 'demand']))
            # forecast_lst.append(moving_avg_demand)
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
            # forecast_lst.append(moving_avg_demand)
        
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]


        # if len(forecast_lst) > day:
        #     avg_lead_time_demand = np.sum(forecast_lst[-day:])
        #     avg_demand = np.sum(demand_lst[-day:])
        #     error_lst.append(avg_demand - avg_lead_time_demand)
        # else:
        #     avg_lead_time_demand = np.sum(forecast_lst[-len(forecast_lst):])
        #     avg_demand = np.sum(demand_lst[-len(demand_lst):])
        #     error_lst.append(avg_demand - avg_lead_time_demand)

        avg_lead_demand =  np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if avg_lead_demand*day >= inventory:
            order_quantity = int(moving_avg_demand*day - inventory)
        else:
            order_quantity = 0
        
        if order_quantity > 0:
            orders.append((period + 4, order_quantity)) 
        
        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [147]:
max_balance = 0
max_day = 0
max_sl = 0

for day in Days_loop:
    print(f'day: {day}')
    balance = China_moving_avg_n_days(data, s, c_c, i, ini_bal, 10, day)
    if balance > max_balance:
        max_balance = balance
        max_day = day
print(f'Current {max_day}, Total Balanced: {max_balance}')

day: 2
day: 3
day: 4
day: 5
day: 6
day: 7
day: 8
day: 9
Current 3, Total Balanced: 4.812582249745737e+47


In [148]:
max_balance_model_2 = max_balance
max_day_model_2 = max_day

max_balance_model_2, max_day_model_2

(4.812582249745737e+47, 3)

### Exponential Smoothing

In [149]:
def China_exp_n_days(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, days):
    cash_balance = initial_cash_balance
    inventory = 0
    orders = []
    # forecast_lst = [0]
    # demand_lst = []
    # error_lst = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        # demand_lst.append(current_demand)
        
        # Calculate cumulative average demand up to the current period
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
            # forecast_lst.append(forecast)
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast
            # forecast_lst.append(forecast)

        avg_lead_demand =  np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        # if len(forecast_lst) > day:
        #     avg_lead_time_demand = np.sum(forecast_lst[-day:])
        #     avg_demand = np.sum(demand_lst[-day:])
        #     error_lst.append(avg_demand - avg_lead_time_demand)
        # else:
        #     avg_lead_time_demand = np.sum(forecast_lst[-len(forecast_lst):])
        #     avg_demand = np.sum(demand_lst[-len(demand_lst):])
        #     error_lst.append(avg_demand - avg_lead_time_demand)

        
        orders_to_receive = [order for order in orders if order[0] == period]
        for order in orders_to_receive:
            inventory += order[1]

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Determine order quantity based
        if avg_lead_demand*days >= inventory:
            order_quantity = int(forecast*days - inventory)
        else:
            order_quantity = 0
        
        if order_quantity > 0:
            orders.append((period + 4, order_quantity)) 
        
        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        # Update inventory with new order
        inventory += order_quantity
    
    return cash_balance



In [157]:
alphas = np.arange(0.01, 0.11, 0.01)

In [158]:
max_balance = 0
max_alpha = 0
max_days = 0

for alpha in alphas:
    print(alpha)
    for day in Days_loop:
        balance = China_exp_n_days(data, s, c_c, i, ini_bal, alpha, day)
        if balance > max_balance:
            max_balance = balance
            max_alpha = alpha
            max_days = day
print(f'Max Alpha {max_alpha}, Max Days {max_days}, Total Balanced: {max_balance}')

0.01
0.02
0.03
0.04
0.05
0.060000000000000005
0.06999999999999999
0.08
0.09
0.09999999999999999
Max Alpha 0.01, Max Days 5, Total Balanced: 5.019634206224602e+47


In [159]:
max_balance_model_3 = max_balance
max_day_model_3 = max_day
max_alpha_model_3 = max_alpha

max_balance_model_3, max_day_model_3, max_alpha_model_3

(5.019634206224602e+47, 3, 0.01)

### Results

In [153]:
print("Cumulative Average Forecast: ${:,.2e}".format(max_balance_model_1))
print("10-Day Moving Average Forecast: ${:,.2e}".format(max_balance_model_2))
print(f"Exponential Smoothing Forecast: ${max_balance_model_3:,.2e}")

Cumulative Average Forecast: $4.80e+47
10-Day Moving Average Forecast: $4.81e+47
Exponential Smoothing Forecast: $5.12e+47
